<a href="https://colab.research.google.com/github/benjawad/AI-for-Environmental-Regulation-Analysis/blob/main/generate_legal_register.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf reportlab PyPDF2 pypdf
!pip install --upgrade transformers accelerate  --quiet
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import fitz  # PyMuPDF
import re
import json
import unicodedata
import pandas as pd
import textwrap
import os
import time


In [ ]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [ ]:
# @title ← 3. Exécutez cette cellule pour charger le modèle "mistralai/Mistral-7B-Instruct-v0.2"

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.1,
    do_sample=True
)


NameError: name 'BitsAndBytesConfig' is not defined

In [ ]:
import fitz  # PyMuPDF
import re
import json
import unicodedata
import textwrap

# --- Fonctions utilitaires ---
def clean_text(text):
    if not text: return ""
    text = unicodedata.normalize("NFKC", text)
    return re.sub(r'\s+', ' ', text).strip()

# --- FONCTION CORRIGÉE ---
def get_page_offset(doc, toc_map):
    """
    Calcule l'offset de manière fiable en testant les premiers actes du sommaire.
    """
    if not toc_map:
        raise ValueError("La table des matières est vide, impossible de calculer l'offset.")

    # On teste les 3 premiers actes du sommaire pour plus de robustesse
    for act_to_find in toc_map[:3]:
        try:
            printed_page_num = act_to_find['start_page_in_doc']

            # Utiliser une partie unique du titre pour la recherche
            search_snippet = clean_text(act_to_find['legal_requirement'].split('portant')[0])
            if len(search_snippet) < 20 or "n°" not in search_snippet:
                search_snippet = clean_text(act_to_find['legal_requirement'])[:60]

            pattern = re.compile(re.escape(search_snippet), re.IGNORECASE)

            for i, page in enumerate(doc):
                page_text = clean_text(page.get_text("text"))
                if pattern.search(page_text):
                    # Si on trouve, on a notre offset et on quitte la fonction
                    return printed_page_num - i
        except Exception:
            # Si un acte échoue, on continue simplement avec le suivant
            continue

    raise ValueError(f"Impossible de localiser les premiers actes du sommaire dans le document pour calculer l'offset.")


def map_table_of_contents_simplified(doc):
    """
    Scanne le texte des premières pages pour trouver les entrées du sommaire.
    """
    toc_map = []
    full_toc_text = ""
    for i in range(min(2, len(doc))):
        full_toc_text += doc[i].get_text("text")

    full_toc_text = clean_text(full_toc_text)

    # Regex amélioré pour capturer les titres, même s'ils contiennent eux-mêmes des points
    pattern = re.compile(
        r"((?:Arrêté|Décision|Décret|Loi|Dahir|Avis)\s+.*?n°\s*[\w\.\-]+.*?)\s*\.{5,}\s*(\d{3,})",
        re.IGNORECASE
    )
    matches = pattern.findall(full_toc_text)

    for match in matches:
        title = clean_text(match[0])
        page_number = int(match[1])
        category_match = re.search(r"^(Arrêté|Décision|Décret|Loi|Dahir|Avis)", title, re.IGNORECASE)
        category = category_match.group(1).capitalize() if category_match else "Inconnu"
        toc_map.append({
            "legal_requirement": title,
            "category": category,
            "start_page_in_doc": page_number,
        })

    toc_map.sort(key=lambda x: x['start_page_in_doc'])

    unique_map = []
    seen_titles = set()
    for item in toc_map:
        if item['legal_requirement'] not in seen_titles:
            unique_map.append(item)
            seen_titles.add(item['legal_requirement'])

    return unique_map

def extract_full_content_from_map(doc, toc_map, page_offset):
    """Extrait le contenu complet en se basant sur la carte du sommaire."""
    final_acts = []
    num_pages_in_doc = len(doc)

    for i, act in enumerate(toc_map):
        start_index = act['start_page_in_doc'] - page_offset

        if i + 1 < len(toc_map):
            next_act_start_page = toc_map[i + 1]['start_page_in_doc']
            end_index = next_act_start_page - page_offset
        else:
            end_index = num_pages_in_doc

        start_index = max(0, start_index)
        end_index = min(num_pages_in_doc, end_index)

        content_parts = []
        if start_index < end_index:
            for page_num in range(start_index, end_index):
                content_parts.append(doc[page_num].get_text("text", sort=True))
        elif 0 <= start_index < num_pages_in_doc:
             content_parts.append(doc[start_index].get_text("text", sort=True))

        full_content = " ".join(content_parts)
        cleaned_content = clean_text(full_content)

        act['full_text_content'] = cleaned_content
        final_acts.append(act)

    return final_acts

# --- SCRIPT PRINCIPAL ---
if __name__ == '__main__':
    pdf_files = [
        'BO_7410_fr.pdf',
    ]

    all_bulletins_data = {}

    for pdf_file_path in pdf_files:
        print(f"\n=================================================")
        print(f"Traitement du fichier : {pdf_file_path}")
        print(f"=================================================")
        try:
            doc = fitz.open(pdf_file_path)

            table_of_contents = map_table_of_contents_simplified(doc)

            if not table_of_contents:
                print("ERREUR : Le sommaire n'a pas pu être parsé pour ce fichier.")
                continue

            print(f"--- Carte du Sommaire ({len(table_of_contents)} actes) ---")
            for item in table_of_contents:
                print(f"  -> Trouvé '{textwrap.shorten(item['legal_requirement'], 100)}' page {item['start_page_in_doc']}")

            # --- LIGNE CORRIGÉE ---
            # On passe la liste entière à la fonction `get_page_offset` qui sait maintenant la gérer.
            page_offset = get_page_offset(doc, table_of_contents)
            print(f"\nDécalage de page : {page_offset}\n")

            final_data = extract_full_content_from_map(doc, table_of_contents, page_offset)

            print(f"--- {len(final_data)} actes extraits ---")
            for act in final_data:
                print(f"  - {act['category']}: {textwrap.shorten(act['legal_requirement'], 100)}")

            all_bulletins_data[pdf_file_path] = final_data

        except FileNotFoundError:
             print(f"ERREUR : Fichier non trouvé '{pdf_file_path}'")
        except Exception as e:
            print(f"Une erreur est survenue lors du traitement de {pdf_file_path} : {e}")
            import traceback
            traceback.print_exc()

    output_filename = 'data_preprocessed_BULLETINS.json'
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(all_bulletins_data, f, indent=2, ensure_ascii=False)
    print(f"\n✅ Toutes les données ont été sauvegardées dans '{output_filename}'")



Traitement du fichier : BO_7410_fr.pdf
--- Carte du Sommaire (14 actes) ---
  -> Trouvé 'Dahir n° 1-25-39 du 12 chaoual 1446 (11 avril 2025), portant nomination de la présidente du [...]' page 2032
  -> Trouvé 'Dahir n° 1-25-40 du 12 chaoual 1446 (11 avril 2025) portant renouvellement de nomination de la [...]' page 2032
  -> Trouvé 'Dahir n° 1-25-41 du 12 chaoual 1446 (11 avril 2025) portant nomination du délégué [...]' page 2032
  -> Trouvé 'Dahir n° 1-25-46 du 17 kaada 1446 (15 mai 2025) portant nomination du directeur général du [...]' page 2032
  -> Trouvé 'Dahir n° 1-25-47 du 17 kaada 1446 (15 mai 2025) portant nomination du directeur général de [...]' page 2032
  -> Trouvé 'Dahir n° 1-24-12 du 10 chaabane 1445 (20 février 2024) portant promulgation de la loi n° 01-22 [...]' page 2033
  -> Trouvé 'Décret n°2-25-366 du 24 kaada 1446 (22 mai 2025) relatif à la navigation aérienne militaire' page 2040
  -> Trouvé 'Décret n° 2-25-269 du 21 kaada 1446 (19 mai 2025) approuvant l’accor

In [ ]:
# --- Étape 2 : Fonctions d'Analyse avec le Bon Template de Prompt ---

def call_llm_mistral(prompt):
    """
    Formate le prompt pour Mistral Instruct, appelle le pipeline local,
    et nettoie la réponse pour extraire le JSON.
    """
    # Template correct pour Mistral Instruct
    formatted_prompt = f"[INST] {prompt} [/INST]"

    try:
        output = text_generator(formatted_prompt)
        raw_response = output[0]['generated_text']
        # Extraire uniquement le texte généré par le modèle après le prompt
        cleaned_response = raw_response.split("[/INST]")[-1].strip()

        # Tentative d'extraire le JSON du texte
        # Le modèle peut parfois ajouter des commentaires avant/après le JSON
        json_match = re.search(r'\{.*\}', cleaned_response, re.DOTALL)
        if json_match:
            return json.loads(json_match.group(0))
        else:
            print(f"  -> Avertissement : Le LLM n'a pas retourné un JSON valide. Réponse brute : {cleaned_response}")
            return None # Retourne None en cas d'échec

    except json.JSONDecodeError:
        print(f"  -> Erreur : Impossible de décoder le JSON de la réponse du LLM. Réponse brute : {cleaned_response}")
        return None
    except Exception as e:
        print(f"  -> Erreur lors de l'appel LLM : {e}")
        return None

def analyze_legal_act(act_data, project_description):
    """
    Analyse un acte juridique avec un prompt CoT finalisé qui insiste
    lourdement sur le format de sortie JSON.
    """
    print(f"\n  -> Analyse de : {textwrap.shorten(act_data['legal_requirement'], 100)}")

    # --- PROMPT FINAL AVEC INSTRUCTION DE FORMATAGE RENFORCÉE ---
    prompt = f"""
    Tu es un assistant expert en conformité réglementaire. Ta tâche est d'analyser un texte de loi et de le classer pour un projet industriel.

    **Contexte du Projet:**
    {project_description}

    **Texte de Loi à Analyser:**
    - Titre : {act_data['legal_requirement']}
    - Contenu :
    \"\"\"
    {textwrap.shorten(act_data['full_text_content'], 4000, placeholder='...')}
    \"\"\"

    **Instructions:**
    Suis ce processus de raisonnement pour arriver à ta réponse finale :
    1.  **Résumé et Pertinence :** D'abord, résume le texte. Ensuite, détermine si le sujet a un impact DIRECT sur une usine chimique (construction, opération, sécurité, environnement, permis).
    2.  **Classification :** Si le texte est pertinent, choisis la catégorie la plus appropriée dans cette liste : ['General Environmental & Sustainability Regulations', 'Solid Waste & auxilary products', 'Water and Liquid discharges', 'Noise and vibrations', 'Energy', 'Air', 'Hazardeous materials', 'Occupational Health and safety']. Si non pertinent, la catégorie est "Non Applicable".
    3.  **Remplissage des Champs :** Remplis tous les champs requis en te basant sur ton analyse. Pour les champs avec des choix contraints, utilise UNIQUEMENT les options fournies.

    **Réponse Finale:**
    Après avoir suivi ces étapes de raisonnement, fournis ta réponse finale. Ta réponse doit être **uniquement et exclusivement un dictionnaire JSON valide**. N'ajoute aucun texte, aucune explication, ni de numérotation avant ou après le JSON.

    **Format JSON Obligatoire:**
    {{
      "category_title": "...",
      "description": "...",
      "phase": "['All project phases', 'Pre-construction', 'Evaluate/Define', 'Construction/Operation']",
      "activity_aspect": "...",
      "impacts": "['Environment', 'Health & Safety', 'Regulatory & Permitting']",
      "type": "['Regulation', 'Guideline', 'Standard']",
      "task": "...",
      "responsibility": "...",
      "comments": "..."
    }}
    """

    # On utilise votre fonction existante pour appeler le modèle
    analysis_result = call_llm_mistral(prompt)
    return analysis_result

In [ ]:
import time
if __name__ == '__main__':

    input_json_path = 'data_preprocessed_BULLETINS.json'

    project_description = """
    Projet de construction d'une usine d'additifs chimiques pour le client NOVADDIX.
    L'usine produira 20 000 tonnes/an d'additifs.
    Domaine : Industriel - Chimie. Classe de produit : Produits inflammables et toxiques.
    """
    print("STARTING")
    try:
        with open(input_json_path, 'r', encoding='utf-8') as f:
            all_bulletins_data = json.load(f)
        print(f"Fichier de données '{input_json_path}' chargé avec succès.")
    except FileNotFoundError:
        print(f"ERREUR : Le fichier d'entrée '{input_json_path}' n'a pas été trouvé.")
        exit()

    all_analyzed_acts = []
    start_time = time.time()

    for pdf_file, legal_acts in all_bulletins_data.items():
        print(f"\n--- Traitement des actes de : {pdf_file} ---")
        for act in legal_acts:
            llm_analysis = analyze_legal_act(act, project_description)

            if llm_analysis:
                # On ajoute les données initiales (parsing) aux données analysées (LLM)
                full_entry = {
                    "Legal Requirement": act['legal_requirement'],
                    "Date": re.search(r'\((\d{1,2}\s+\w+\s+\d{4})\)', act['legal_requirement']).group(1) if re.search(r'\((\d{1,2}\s+\w+\s+\d{4})\)', act['legal_requirement']) else 'N/A',
                    "Type": act['category'],
                    "Jurisdiction": "National",
                    "CategoryTitle": llm_analysis.get("category_title", "Uncategorized"), # NOUVELLE CLÉ POUR LE GROUPEMENT
                    "Description": llm_analysis.get("description", "N/A"),
                    "Phase": llm_analysis.get("phase", "N/A"),
                    "Activity/Aspect": llm_analysis.get("activity_aspect", "N/A"),
                    "Impacts": llm_analysis.get("impacts", "N/A"),
                    "Task": llm_analysis.get("task", "N/A"),
                    "Responsibility": llm_analysis.get("responsibility", "N/A"),
                    "Comments": llm_analysis.get("comments", "N/A"),
                }
                all_analyzed_acts.append(full_entry)
            time.sleep(1)

    end_time = time.time()
    print(f"\n--- Analyse LLM terminée en {end_time - start_time:.2f} secondes ---")

    if all_analyzed_acts:
        # Création du DataFrame avec toutes les données
        df = pd.DataFrame(all_analyzed_acts)
        try:
            output_csv_file = 'registre_legal_output.csv'
            df.to_csv(output_csv_file, index=False, encoding='utf-8-sig')
            print(f"\nFichier CSV généré avec succès : '{output_csv_file}'")
        except Exception as e:
            print(f"\nErreur lors de la génération du fichier CSV : {e}")

        structured_data_for_pdf = []
        # On groupe par la nouvelle clé 'CategoryTitle' générée par le LLM
        grouped = df.groupby('CategoryTitle')

        for category_name, group in grouped:
            category_dict = {
                "category_title": category_name,
                "rows": []
            }

            for index, row in group.iterrows():
                # Créer la liste de chaînes dans le bon ordre pour le PDF
                row_list = [
                    row['Phase'], row['Activity/Aspect'], row['Impacts'],
                    row['Jurisdiction'], row['Type'], row['Legal Requirement'],
                    row['Date'], row['Description'], row['Task'],
                    row['Responsibility'], row['Comments']
                ]
                category_dict["rows"].append(row_list)

            structured_data_for_pdf.append(category_dict)

        print(f"\nDonnées transformées en {len(structured_data_for_pdf)} catégories pour le PDF.")
    else:
        print("\nAucune donnée n'a été générée pour le registre légal final.")

In [ ]:
import os
import pandas as pd
from reportlab.platypus  import SimpleDocTemplate, Table, TableStyle, Paragraph
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.pagesizes import landscape, A3
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER, TA_LEFT

# --- FONCTION DE GÉNÉRATION DE PDF (maintenant elle accepte les données) ---

def generate_legal_register_pdf(structured_data, output_filename="legal_register_final.pdf"):
    """
    Génère un PDF à partir d'une structure de données dynamique passée en paramètre.
    """
    # --- Document Setup ---
    pagesize = landscape(A3)
    margin = 1.0 * inch
    doc_width, doc_height = pagesize
    table_width = doc_width - 2 * margin

    doc = SimpleDocTemplate(
        output_filename,
        pagesize=pagesize,
        rightMargin=margin, leftMargin=margin,
        topMargin=1.5 * inch, bottomMargin=margin,
    )

    # --- Styles ---
    styles = getSampleStyleSheet()
    header_style = ParagraphStyle(name='HeaderStyle', parent=styles['Normal'], fontName='Helvetica-Bold', fontSize=5, leading=8, alignment=TA_CENTER, textColor=colors.white)
    category_header_style = ParagraphStyle(name='CategoryHeaderStyle', parent=header_style, fontSize=6, leading=10, alignment=TA_LEFT) # Alignement à gauche pour les titres
    cell_style = ParagraphStyle(name='CellStyle', parent=styles['Normal'], fontName='Helvetica', alignment=TA_LEFT, fontSize=5, leading=7)
    bold_cell_style = ParagraphStyle(name='BoldCellStyle', parent=cell_style, fontName='Helvetica-Bold')

    # --- Fonctions pour créer les paragraphes ---
    def create_header_paragraph(text): return Paragraph(str(text).replace('\n', '<br/>'), header_style)
    def create_category_paragraph(text): return Paragraph(str(text).replace('\n', '<br/>'), category_header_style)
    def create_paragraph(text, style=cell_style): return Paragraph(str(text).replace('\n', '<br/>'), style)

    # --- Assemblage Dynamique de la Table ---
    header_row_str = ['Phase', 'Activity/Aspect', 'Impacts', 'Jurisdiction', 'Type', 'Legal Requirement', 'Date', 'Description', 'Task', 'Responsibility', 'Comments']
    table_data = [[create_header_paragraph(h) for h in header_row_str]]

    red_header_indices = []

    # La fonction utilise maintenant les données passées en paramètre
    for category in structured_data:
        category_title = category.get("category_title", "Uncategorized")

        red_header_indices.append(len(table_data))
        red_header_row = [create_category_paragraph(category_title)] + [''] * (len(header_row_str) - 1)
        table_data.append(red_header_row)

        for row_data in category.get("rows", []):
            processed_row = [
                create_paragraph(row_data[0], bold_cell_style),
                *[create_paragraph(cell) for cell in row_data[1:]]
            ]
            table_data.append(processed_row)

    # --- Largeurs de Colonnes ---
    col_widths_proportions = [0.07, 0.08, 0.07, 0.05, 0.05, 0.15, 0.06, 0.15, 0.12, 0.07, 0.13]
    col_widths = [p * table_width for p in col_widths_proportions]

    # --- Style de la Table ---
    style_commands = [
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#002060')),
        # Padding
        ('TOP_PADDING', (0, 0), (-1, -1), 3),
        ('BOTTOM_PADDING', (0, 0), (-1, -1), 3),
        ('LEFT_PADDING', (0, 0), (-1, -1), 3),
        ('RIGHT_PADDING', (0, 0), (-1, -1), 3),
    ]
    for row_idx in red_header_indices:
        style_commands.append(('SPAN', (0, row_idx), (-1, row_idx)))
        style_commands.append(('BACKGROUND', (0, row_idx), (-1, row_idx), colors.HexColor('#C00000')))

    # --- Création et Sauvegarde du PDF ---
    table = Table(table_data, colWidths=col_widths)
    table.setStyle(TableStyle(style_commands))

    doc.build([table])
    print(f"PDF '{output_filename}' généré avec succès.")
    print(f"Fichier sauvegardé à : {os.path.abspath(output_filename)}")


def main():
    """
    Orchestre le chargement des données CSV, leur transformation en structure
    pour le PDF, puis appelle le générateur de PDF.
    """
    input_csv_path = 'registre_legal_output.csv'

    try:
        # MODIFICATION 1 : Le séparateur est maintenant une virgule ','
        df = pd.read_csv(input_csv_path, sep=',')
        df.fillna("", inplace=True)
        print(f"{len(df)} exigences légales chargées depuis '{input_csv_path}'.")
    except FileNotFoundError:
        print(f"ERREUR : Le fichier CSV '{input_csv_path}' n'a pas été trouvé.")
        return

    # 2. Transformer les données en la structure attendue par la fonction PDF
    structured_data_from_llm = []

    # Définir l'ordre souhaité des catégories
    category_order = [
        'General Environmental & Sustainability Regulations',
        'Solid Waste & auxilary products',
        'Water and Liquid discharges',
        'Noise and vibrations',
        'Energy',
        'Air',
        # MODIFICATION 2 : Correction de la faute de frappe pour correspondre au CSV
        'Hazardous materials',
        'Occupational Health and safety',
        'Réglementaire & Permis',
    ]

    # MODIFICATION 3 : Nous utilisons 'CategoryTitle' pour le regroupement, pas 'Impacts'
    grouping_column = 'CategoryTitle'

    if grouping_column not in df.columns:
        print(f"ERREUR: La colonne de regroupement '{grouping_column}' n'existe pas dans le CSV.")
        print(f"Colonnes disponibles : {list(df.columns)}")
        return

    # On convertit la colonne de groupement en type Catégorie pour respecter l'ordre
    df[grouping_column] = pd.Categorical(df[grouping_column], categories=category_order, ordered=True)
    df = df.sort_values(grouping_column)

    # MODIFICATION 4 : Regroupement par 'CategoryTitle'
    grouped = df.groupby(grouping_column, observed=False)

    for category_name, group in grouped:
        category_dict = {
            "category_title": category_name,
            "rows": []
        }

        for index, row in group.iterrows():
            # Assurez-vous que l'ordre des colonnes ici correspond à 'header_row_str'
            row_list = [
                row.get('Phase', ''),
                row.get('Activity/Aspect', ''),
                row.get('Impacts', ''), # La colonne 'Impacts' est toujours affichée, mais pas utilisée pour grouper
                row.get('Jurisdiction', ''),
                row.get('Type', ''),
                row.get('Legal Requirement', ''),
                row.get('Date', ''),
                row.get('Description', ''),
                row.get('Task', ''),
                row.get('Responsibility', ''),
                row.get('Comments', '')
            ]
            category_dict["rows"].append(row_list)

        # On n'ajoute la catégorie que si elle contient des lignes
        if category_dict["rows"]:
            structured_data_from_llm.append(category_dict)

    print(f"Données transformées en {len(structured_data_from_llm)} catégories pour le PDF.")

    # 3. Appeler la fonction de génération de PDF avec les données réelles
    generate_legal_register_pdf(structured_data_from_llm)

# --- Lancement du script ---
if __name__ == "__main__":
    main()

In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import fitz  # PyMuPDF
import re
import json
import unicodedata
import pandas as pd
import textwrap
import os
import time
import tempfile
import shutil
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.pagesizes import landscape, A3
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER, TA_LEFT

# Global variables for the model
text_generator = None
model_loaded = False

# --- Your existing functions (keeping them exactly as they are) ---

def clean_text(text):
    if not text: return ""
    text = unicodedata.normalize("NFKC", text)
    return re.sub(r'\s+', ' ', text).strip()

def get_page_offset(doc, toc_map):
    """
    Calcule l'offset de manière fiable en testant les premiers actes du sommaire.
    """
    if not toc_map:
        raise ValueError("La table des matières est vide, impossible de calculer l'offset.")

    # On teste les 3 premiers actes du sommaire pour plus de robustesse
    for act_to_find in toc_map[:3]:
        try:
            printed_page_num = act_to_find['start_page_in_doc']

            # Utiliser une partie unique du titre pour la recherche
            search_snippet = clean_text(act_to_find['legal_requirement'].split('portant')[0])
            if len(search_snippet) < 20 or "n°" not in search_snippet:
                search_snippet = clean_text(act_to_find['legal_requirement'])[:60]

            pattern = re.compile(re.escape(search_snippet), re.IGNORECASE)

            for i, page in enumerate(doc):
                page_text = clean_text(page.get_text("text"))
                if pattern.search(page_text):
                    # Si on trouve, on a notre offset et on quitte la fonction
                    return printed_page_num - i
        except Exception:
            # Si un acte échoue, on continue simplement avec le suivant
            continue

    raise ValueError(f"Impossible de localiser les premiers actes du sommaire dans le document pour calculer l'offset.")

def map_table_of_contents_simplified(doc):
    """
    Scanne le texte des premières pages pour trouver les entrées du sommaire.
    """
    toc_map = []
    full_toc_text = ""
    for i in range(min(2, len(doc))):
        full_toc_text += doc[i].get_text("text")

    full_toc_text = clean_text(full_toc_text)

    # Regex amélioré pour capturer les titres, même s'ils contiennent eux-mêmes des points
    pattern = re.compile(
        r"((?:Arrêté|Décision|Décret|Loi|Dahir|Avis)\s+.*?n°\s*[\w\.\-]+.*?)\s*\.{5,}\s*(\d{3,})",
        re.IGNORECASE
    )
    matches = pattern.findall(full_toc_text)

    for match in matches:
        title = clean_text(match[0])
        page_number = int(match[1])
        category_match = re.search(r"^(Arrêté|Décision|Décret|Loi|Dahir|Avis)", title, re.IGNORECASE)
        category = category_match.group(1).capitalize() if category_match else "Inconnu"
        toc_map.append({
            "legal_requirement": title,
            "category": category,
            "start_page_in_doc": page_number,
        })

    toc_map.sort(key=lambda x: x['start_page_in_doc'])

    unique_map = []
    seen_titles = set()
    for item in toc_map:
        if item['legal_requirement'] not in seen_titles:
            unique_map.append(item)
            seen_titles.add(item['legal_requirement'])

    return unique_map

def extract_full_content_from_map(doc, toc_map, page_offset):
    """Extrait le contenu complet en se basant sur la carte du sommaire."""
    final_acts = []
    num_pages_in_doc = len(doc)

    for i, act in enumerate(toc_map):
        start_index = act['start_page_in_doc'] - page_offset

        if i + 1 < len(toc_map):
            next_act_start_page = toc_map[i + 1]['start_page_in_doc']
            end_index = next_act_start_page - page_offset
        else:
            end_index = num_pages_in_doc

        start_index = max(0, start_index)
        end_index = min(num_pages_in_doc, end_index)

        content_parts = []
        if start_index < end_index:
            for page_num in range(start_index, end_index):
                content_parts.append(doc[page_num].get_text("text", sort=True))
        elif 0 <= start_index < num_pages_in_doc:
             content_parts.append(doc[start_index].get_text("text", sort=True))

        full_content = " ".join(content_parts)
        cleaned_content = clean_text(full_content)

        act['full_text_content'] = cleaned_content
        final_acts.append(act)

    return final_acts

def call_llm_mistral(prompt):
    """
    Formate le prompt pour Mistral Instruct, appelle le pipeline local,
    et nettoie la réponse pour extraire le JSON.
    """
    global text_generator
    if text_generator is None:
        return {"error": "Model not loaded"}

    # Template correct pour Mistral Instruct
    formatted_prompt = f"[INST] {prompt} [/INST]"

    try:
        output = text_generator(formatted_prompt)
        raw_response = output[0]['generated_text']
        # Extraire uniquement le texte généré par le modèle après le prompt
        cleaned_response = raw_response.split("[/INST]")[-1].strip()

        # Tentative d'extraire le JSON du texte
        # Le modèle peut parfois ajouter des commentaires avant/après le JSON
        json_match = re.search(r'\{.*\}', cleaned_response, re.DOTALL)
        if json_match:
            return json.loads(json_match.group(0))
        else:
            print(f"  -> Avertissement : Le LLM n'a pas retourné un JSON valide. Réponse brute : {cleaned_response}")
            return None # Retourne None en cas d'échec

    except json.JSONDecodeError:
        print(f"  -> Erreur : Impossible de décoder le JSON de la réponse du LLM. Réponse brute : {cleaned_response}")
        return None
    except Exception as e:
        print(f"  -> Erreur lors de l'appel LLM : {e}")
        return None

def analyze_legal_act(act_data, project_description):
    """
    Analyse un acte juridique avec un prompt CoT finalisé qui insiste
    lourdement sur le format de sortie JSON.
    """
    print(f"\n  -> Analyse de : {textwrap.shorten(act_data['legal_requirement'], 100)}")

    # --- PROMPT FINAL AVEC INSTRUCTION DE FORMATAGE RENFORCÉE ---
    prompt = f"""
    Tu es un assistant expert en conformité réglementaire. Ta tâche est d'analyser un texte de loi et de le classer pour un projet industriel.

    **Contexte du Projet:**
    {project_description}

    **Texte de Loi à Analyser:**
    - Titre : {act_data['legal_requirement']}
    - Contenu :
    \"\"\"
    {textwrap.shorten(act_data['full_text_content'], 4000, placeholder='...')}
    \"\"\"

    **Instructions:**
    Suis ce processus de raisonnement pour arriver à ta réponse finale :
    1.  **Résumé et Pertinence :** D'abord, résume le texte. Ensuite, détermine si le sujet a un impact DIRECT sur une usine chimique (construction, opération, sécurité, environnement, permis).
    2.  **Classification :** Si le texte est pertinent, choisis la catégorie la plus appropriée dans cette liste : ['General Environmental & Sustainability Regulations', 'Solid Waste & auxilary products', 'Water and Liquid discharges', 'Noise and vibrations', 'Energy', 'Air', 'Hazardeous materials', 'Occupational Health and safety']. Si non pertinent, la catégorie est "Non Applicable".
    3.  **Remplissage des Champs :** Remplis tous les champs requis en te basant sur ton analyse. Pour les champs avec des choix contraints, utilise UNIQUEMENT les options fournies.

    **Réponse Finale:**
    Après avoir suivi ces étapes de raisonnement, fournis ta réponse finale. Ta réponse doit être **uniquement et exclusivement un dictionnaire JSON valide**. N'ajoute aucun texte, aucune explication, ni de numérotation avant ou après le JSON.

    **Format JSON Obligatoire:**
    {{
      "category_title": "...",
      "description": "...",
      "phase": "['All project phases', 'Pre-construction', 'Evaluate/Define', 'Construction/Operation']",
      "activity_aspect": "...",
      "impacts": "['Environment', 'Health & Safety', 'Regulatory & Permitting']",
      "type": "['Regulation', 'Guideline', 'Standard']",
      "task": "...",
      "responsibility": "...",
      "comments": "..."
    }}
    """

    # On utilise votre fonction existante pour appeler le modèle
    analysis_result = call_llm_mistral(prompt)
    return analysis_result

def generate_legal_register_pdf(structured_data, output_filename="legal_register_final.pdf"):
    """
    Génère un PDF à partir d'une structure de données dynamique passée en paramètre.
    """
    # --- Document Setup ---
    pagesize = landscape(A3)
    margin = 1.0 * inch
    doc_width, doc_height = pagesize
    table_width = doc_width - 2 * margin

    doc = SimpleDocTemplate(
        output_filename,
        pagesize=pagesize,
        rightMargin=margin, leftMargin=margin,
        topMargin=1.5 * inch, bottomMargin=margin,
    )

    # --- Styles ---
    styles = getSampleStyleSheet()
    header_style = ParagraphStyle(name='HeaderStyle', parent=styles['Normal'], fontName='Helvetica-Bold', fontSize=5, leading=8, alignment=TA_CENTER, textColor=colors.white)
    category_header_style = ParagraphStyle(name='CategoryHeaderStyle', parent=header_style, fontSize=6, leading=10, alignment=TA_LEFT) # Alignement à gauche pour les titres
    cell_style = ParagraphStyle(name='CellStyle', parent=styles['Normal'], fontName='Helvetica', alignment=TA_LEFT, fontSize=5, leading=7)
    bold_cell_style = ParagraphStyle(name='BoldCellStyle', parent=cell_style, fontName='Helvetica-Bold')

    # --- Fonctions pour créer les paragraphes ---
    def create_header_paragraph(text): return Paragraph(str(text).replace('\n', '<br/>'), header_style)
    def create_category_paragraph(text): return Paragraph(str(text).replace('\n', '<br/>'), category_header_style)
    def create_paragraph(text, style=cell_style): return Paragraph(str(text).replace('\n', '<br/>'), style)

    # --- Assemblage Dynamique de la Table ---
    header_row_str = ['Phase', 'Activity/Aspect', 'Impacts', 'Jurisdiction', 'Type', 'Legal Requirement', 'Date', 'Description', 'Task', 'Responsibility', 'Comments']
    table_data = [[create_header_paragraph(h) for h in header_row_str]]

    red_header_indices = []

    # La fonction utilise maintenant les données passées en paramètre
    for category in structured_data:
        category_title = category.get("category_title", "Uncategorized")

        red_header_indices.append(len(table_data))
        red_header_row = [create_category_paragraph(category_title)] + [''] * (len(header_row_str) - 1)
        table_data.append(red_header_row)

        for row_data in category.get("rows", []):
            processed_row = [
                create_paragraph(row_data[0], bold_cell_style),
                *[create_paragraph(cell) for cell in row_data[1:]]
            ]
            table_data.append(processed_row)

    # --- Largeurs de Colonnes ---
    col_widths_proportions = [0.07, 0.08, 0.07, 0.05, 0.05, 0.15, 0.06, 0.15, 0.12, 0.07, 0.13]
    col_widths = [p * table_width for p in col_widths_proportions]

    # --- Style de la Table ---
    style_commands = [
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#002060')),
        # Padding
        ('TOP_PADDING', (0, 0), (-1, -1), 3),
        ('BOTTOM_PADDING', (0, 0), (-1, -1), 3),
        ('LEFT_PADDING', (0, 0), (-1, -1), 3),
        ('RIGHT_PADDING', (0, 0), (-1, -1), 3),
    ]
    for row_idx in red_header_indices:
        style_commands.append(('SPAN', (0, row_idx), (-1, row_idx)))
        style_commands.append(('BACKGROUND', (0, row_idx), (-1, row_idx), colors.HexColor('#C00000')))

    # --- Création et Sauvegarde du PDF ---
    table = Table(table_data, colWidths=col_widths)
    table.setStyle(TableStyle(style_commands))

    doc.build([table])
    print(f"PDF '{output_filename}' généré avec succès.")
    print(f"Fichier sauvegardé à : {os.path.abspath(output_filename)}")

# --- Gradio Functions ---

def load_model():
    """Load the Mistral model"""
    global text_generator, model_loaded

    if model_loaded:
        return "✅ Model already loaded!"

    try:
        model_id = "mistralai/Mistral-7B-Instruct-v0.2"

        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )

        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto"
        )

        text_generator = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            temperature=0.1,
            do_sample=True
        )

        model_loaded = True
        return "✅ Model loaded successfully!"

    except Exception as e:
        return f"❌ Error loading model: {str(e)}"

def process_pdf(pdf_file, project_description, progress=gr.Progress()):
    """Main processing function for Gradio"""
    global text_generator

    if not model_loaded or text_generator is None:
        return None, None, "❌ Please load the model first!"

    if pdf_file is None:
        return None, None, "❌ Please upload a PDF file!"

    if not project_description.strip():
        project_description = """
        Projet de construction d'une usine d'additifs chimiques pour le client NOVADDIX.
        L'usine produira 20 000 tonnes/an d'additifs.
        Domaine : Industriel - Chimie. Classe de produit : Produits inflammables et toxiques.
        """

    try:
        progress(0.1, desc="Opening PDF...")

        # Create temporary directory for processing
        temp_dir = tempfile.mkdtemp()

        # Copy uploaded file to temp directory
        temp_pdf_path = os.path.join(temp_dir, "uploaded_file.pdf")
        shutil.copy(pdf_file.name, temp_pdf_path)

        # Process PDF
        doc = fitz.open(temp_pdf_path)

        progress(0.2, desc="Extracting table of contents...")

        table_of_contents = map_table_of_contents_simplified(doc)

        if not table_of_contents:
            return None, None, "❌ Could not parse table of contents from the PDF."

        progress(0.3, desc="Calculating page offset...")

        page_offset = get_page_offset(doc, table_of_contents)

        progress(0.4, desc="Extracting content...")

        final_data = extract_full_content_from_map(doc, table_of_contents, page_offset)

        # Save preprocessed data
        preprocessed_path = os.path.join(temp_dir, 'data_preprocessed_BULLETINS.json')
        all_bulletins_data = {os.path.basename(temp_pdf_path): final_data}

        with open(preprocessed_path, 'w', encoding='utf-8') as f:
            json.dump(all_bulletins_data, f, indent=2, ensure_ascii=False)

        # Analyze with LLM
        all_analyzed_acts = []
        total_acts = len(final_data)

        for i, act in enumerate(final_data):
            progress(0.5 + (i / total_acts) * 0.4, desc=f"Analyzing act {i+1}/{total_acts}...")

            llm_analysis = analyze_legal_act(act, project_description)

            if llm_analysis:
                full_entry = {
                    "Legal Requirement": act['legal_requirement'],
                    "Date": re.search(r'\((\d{1,2}\s+\w+\s+\d{4})\)', act['legal_requirement']).group(1) if re.search(r'\((\d{1,2}\s+\w+\s+\d{4})\)', act['legal_requirement']) else 'N/A',
                    "Type": act['category'],
                    "Jurisdiction": "National",
                    "CategoryTitle": llm_analysis.get("category_title", "Uncategorized"),
                    "Description": llm_analysis.get("description", "N/A"),
                    "Phase": llm_analysis.get("phase", "N/A"),
                    "Activity/Aspect": llm_analysis.get("activity_aspect", "N/A"),
                    "Impacts": llm_analysis.get("impacts", "N/A"),
                    "Task": llm_analysis.get("task", "N/A"),
                    "Responsibility": llm_analysis.get("responsibility", "N/A"),
                    "Comments": llm_analysis.get("comments", "N/A"),
                }
                all_analyzed_acts.append(full_entry)

            time.sleep(0.5)  # Small delay to avoid overwhelming the model

        progress(0.9, desc="Generating outputs...")

        # Create CSV
        if all_analyzed_acts:
            df = pd.DataFrame(all_analyzed_acts)
            csv_path = os.path.join(temp_dir, 'registre_legal_output.csv')
            df.to_csv(csv_path, index=False, encoding='utf-8-sig')

            # Prepare data for PDF
            structured_data_for_pdf = []

            # Define category order
            category_order = [
                'General Environmental & Sustainability Regulations',
                'Solid Waste & auxilary products',
                'Water and Liquid discharges',
                'Noise and vibrations',
                'Energy',
                'Air',
                'Hazardous materials',
                'Occupational Health and safety',
            ]

            # Group by CategoryTitle
            df['CategoryTitle'] = pd.Categorical(df['CategoryTitle'], categories=category_order, ordered=True)
            df = df.sort_values('CategoryTitle')
            grouped = df.groupby('CategoryTitle', observed=False)

            for category_name, group in grouped:
                if len(group) > 0:  # Only add if there are rows
                    category_dict = {
                        "category_title": category_name,
                        "rows": []
                    }

                    for index, row in group.iterrows():
                        row_list = [
                            row.get('Phase', ''),
                            row.get('Activity/Aspect', ''),
                            row.get('Impacts', ''),
                            row.get('Jurisdiction', ''),
                            row.get('Type', ''),
                            row.get('Legal Requirement', ''),
                            row.get('Date', ''),
                            row.get('Description', ''),
                            row.get('Task', ''),
                            row.get('Responsibility', ''),
                            row.get('Comments', '')
                        ]
                        category_dict["rows"].append(row_list)

                    structured_data_for_pdf.append(category_dict)

            # Generate PDF
            pdf_path = os.path.join(temp_dir, 'legal_register_final.pdf')
            generate_legal_register_pdf(structured_data_for_pdf, pdf_path)

            progress(1.0, desc="Complete!")

            # Copy files to a location where Gradio can access them
            final_csv_path = os.path.join(os.getcwd(), f"output_{int(time.time())}_registre_legal.csv")
            final_pdf_path = os.path.join(os.getcwd(), f"output_{int(time.time())}_legal_register.pdf")

            shutil.copy(csv_path, final_csv_path)
            shutil.copy(pdf_path, final_pdf_path)

            success_message = f"✅ Processing complete!\n- {len(table_of_contents)} acts found in table of contents\n- {len(all_analyzed_acts)} acts successfully analyzed\n- Files generated: CSV and PDF"

            # Cleanup
            doc.close()
            shutil.rmtree(temp_dir)

            return final_csv_path, final_pdf_path, success_message
        else:
            return None, None, "❌ No acts were successfully analyzed."

    except Exception as e:
        if 'doc' in locals():
            doc.close()
        if 'temp_dir' in locals():
            shutil.rmtree(temp_dir, ignore_errors=True)
        return None, None, f"❌ Error processing PDF: {str(e)}"

# --- Gradio Interface ---

def create_interface():
    with gr.Blocks(title="Legal Register PDF Processor", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🏛️ Legal Register PDF Processor")
        gr.Markdown("Upload a PDF bulletin and generate a legal compliance register with AI analysis.")

        with gr.Row():
            with gr.Column():
                gr.Markdown("## Step 1: Load Model")
                load_btn = gr.Button("🤖 Load Mistral Model", variant="primary")
                model_status = gr.Textbox(label="Model Status", interactive=False)

                gr.Markdown("## Step 2: Upload PDF & Configure")
                pdf_input = gr.File(
                    label="Upload PDF Bulletin",
                    file_types=[".pdf"],
                    type="filepath"
                )

                project_desc = gr.Textbox(
                    label="Project Description",
                    placeholder="Describe your industrial project...",
                    lines=4,
                    value="""Projet de construction d'une usine d'additifs chimiques pour le client NOVADDIX.
L'usine produira 20 000 tonnes/an d'additifs.
Domaine : Industriel - Chimie. Classe de produit : Produits inflammables et toxiques."""
                )

                process_btn = gr.Button("🚀 Process PDF", variant="primary", size="lg")

        with gr.Row():
            with gr.Column():
                gr.Markdown("## Results")
                status_output = gr.Textbox(label="Processing Status", lines=3)

                with gr.Row():
                    csv_download = gr.File(label="📊 Download CSV")
                    pdf_download = gr.File(label="📋 Download PDF Report")

        # Event handlers
        load_btn.click(
            fn=load_model,
            outputs=[model_status]
        )

        process_btn.click(
            fn=process_pdf,
            inputs=[pdf_input, project_desc],
            outputs=[csv_download, pdf_download, status_output]
        )

        # Examples section
        gr.Markdown("## 📖 Instructions")
        gr.Markdown("""
        1. **Load the Model**: Click the "Load Mistral Model" button and wait for confirmation
        2. **Upload PDF**: Select a legal bulletin PDF file
        3. **Configure Project**: Edit the project description if needed
        4. **Process**: Click "Process PDF" and wait for the analysis to complete
        5. **Download**: Get your CSV data and PDF report

        **Note**: The first model load may take several minutes. Processing time depends on the number of legal acts in your PDF.
        """)

    return demo

if __name__ == "__main__":
    demo = create_interface()
    demo.launch(share=True, debug=True)